# Creacion de las Variables 'r', 'f' y 'm'

In [1]:
%load_ext watermark
%watermark

2019-07-15T14:22:17+02:00

CPython 3.6.8
IPython 6.2.1

compiler   : GCC 7.3.0
system     : Linux
release    : 4.13.0-46-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit


## Carga de los datos 

In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df = pd.read_pickle('../Data/01.-Carga-de-Datos-y-Exploracion.pkl')

df.head()

,InvoiceNo,InvoiceDate,CustomerID,Total
0,536365,2010-12-01,17850.0,139.12
1,536368,2010-12-01,13047.0,70.05
2,536369,2010-12-01,13047.0,17.85
3,536370,2010-12-01,12583.0,855.86
4,536523,2010-12-01,12868.0,203.30


## Notas 

* Quedarme con el último año.
* Establecer el día del estudio (último día + 1).
* Crear el DataFrame con las variables r, f y m.

## Creación del DataFrame df_rfm 

In [4]:
df.dtypes

InvoiceNo              object
InvoiceDate    datetime64[ns]
CustomerID            float64
Total                 float64
dtype: object

In [5]:
df.tail()

,InvoiceNo,InvoiceDate,CustomerID,Total
22216,581582,2011-12-09,17581.0,29.88
22217,581583,2011-12-09,13777.0,124.60
22218,581584,2011-12-09,13777.0,140.64
22219,581571,2011-12-09,15311.0,494.60
22220,C581569,2011-12-09,17315.0,-7.50


In [6]:
df['InvoiceDate'].max() - df['InvoiceDate'].min()

Timedelta('373 days 00:00:00')

In [7]:
df = df[df['InvoiceDate'].max() - df['InvoiceDate'] <= datetime.timedelta(days = 365)]

df.head()

,InvoiceNo,InvoiceDate,CustomerID,Total
770,C538084,2010-12-09,12649.0,-19.80
771,C538089,2010-12-09,18230.0,-6.75
772,C538114,2010-12-09,14217.0,-16.95
773,C538088,2010-12-09,14766.0,-20.40
774,C538083,2010-12-09,15750.0,-14.25


In [8]:
df['InvoiceDate'].max() - df['InvoiceDate'].min()

Timedelta('365 days 00:00:00')

In [9]:
#now = df['InvoiceDate'].max() + timedelta(days = 1)

now = df['InvoiceDate'].max() + datetime.timedelta(days = 1)

now

Timestamp('2011-12-10 00:00:00')

In [10]:
def process(df_temp):
    recency = (now - df_temp['InvoiceDate'].max()).days
    frequency = len(df_temp)
    monetary = df_temp['Total'].sum()
    return pd.Series([recency,frequency,monetary])

In [11]:
df_rfm = df.groupby('CustomerID').apply(process)

df_rfm.head()

,0,1,2
CustomerID,,,
12346.0,326.0,2.0,0.00
12347.0,3.0,6.0,3598.21
12348.0,76.0,4.0,1797.24
12349.0,19.0,1.0,1757.55
12350.0,311.0,1.0,334.40


In [12]:
df_rfm.columns = ['recency','frequency','monetary']
df_rfm = df_rfm.reset_index()

df_rfm.head()

,CustomerID,recency,frequency,monetary
0,12346.0,326.0,2.0,0.00
1,12347.0,3.0,6.0,3598.21
2,12348.0,76.0,4.0,1797.24
3,12349.0,19.0,1.0,1757.55
4,12350.0,311.0,1.0,334.40


In [13]:
df_rfm.dtypes

CustomerID    float64
recency       float64
frequency     float64
monetary      float64
dtype: object

## Cuartiles

In [14]:
print('El número de clientes es: {}'.format(len(df_rfm)))

El número de clientes es: 4307


In [15]:
quartiles = df_rfm[['recency','frequency','monetary']].quantile([.25,.5,.75]).to_dict()

quartiles

{'frequency': {0.25: 1.0, 0.5: 3.0, 0.75: 5.0},
 'monetary': {0.25: 291.025, 0.5: 644.24, 0.75: 1592.12},
 'recency': {0.25: 17.0, 0.5: 48.0, 0.75: 135.0}}

In [16]:
def r_score(x):
    if x <= quartiles['recency'][.25]:
        return 4
    elif x <= quartiles['recency'][.5]:
        return 3
    elif x <= quartiles['recency'][.75]:
        return 2
    else:
        return 1
    
def f_and_m_score(x,f_or_m):
    if x <= quartiles[f_or_m][.25]:
        return 1
    elif x <= quartiles[f_or_m][.5]:
        return 2
    elif x <= quartiles[f_or_m][.75]:
        return 3
    else:
        return 4

In [17]:
df_rfm['r_score'] = df_rfm['recency'].apply(lambda x: r_score(x))
df_rfm['f_score'] = df_rfm['frequency'].apply(lambda x: f_and_m_score(x,'frequency'))
df_rfm['m_score'] = df_rfm['monetary'].apply(lambda x: f_and_m_score(x,'monetary'))

df_rfm['rfm_score'] = df_rfm['r_score'].map(str) + df_rfm['f_score'].map(str) + df_rfm['m_score'].map(str)

df_rfm.head()

,CustomerID,recency,frequency,monetary,r_score,f_score,m_score,rfm_score
0,12346.0,326.0,2.0,0.00,1,2,1,121
1,12347.0,3.0,6.0,3598.21,4,4,4,444
2,12348.0,76.0,4.0,1797.24,2,3,4,234
3,12349.0,19.0,1.0,1757.55,3,1,4,314
4,12350.0,311.0,1.0,334.40,1,1,2,112


In [18]:
df_rfm.dtypes

CustomerID    float64
recency       float64
frequency     float64
monetary      float64
r_score         int64
f_score         int64
m_score         int64
rfm_score      object
dtype: object

In [19]:
type(df_rfm.loc[0,'rfm_score'])

str

In [20]:
segmentation_map = {
    r'444':'Best Customers',
    r'44[1-3]':'Loyal Customers',
    r'4[2-3][1-4]':'Promising',
    r'41[1-4]':'New Costumers',
    r'3[1-4][1-4]':'Need Atention',
    r'2[1-4][1-4]':'Almost Lost',
    r'1[1-4][1-4]':'Lost'
}

df_rfm['Segment'] = df_rfm['rfm_score'].replace(segmentation_map,regex = True)

df_rfm.head()

,CustomerID,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,Segment
0,12346.0,326.0,2.0,0.00,1,2,1,121,Lost
1,12347.0,3.0,6.0,3598.21,4,4,4,444,Best Customers
2,12348.0,76.0,4.0,1797.24,2,3,4,234,Almost Lost
3,12349.0,19.0,1.0,1757.55,3,1,4,314,Need Atention
4,12350.0,311.0,1.0,334.40,1,1,2,112,Lost


In [21]:
df_rfm[df_rfm['Segment'].apply(len) == 3]

,CustomerID,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,Segment


Por tanto, todos los clientes están segmentados.

## Creación archivo pkl

In [22]:
df.to_pickle('../Data/02.-Creacion-de-df_rfm-y-cuartiles.pkl')

In [23]:
!ls -l --block-size=M ../Data/02.-Creacion-de-df_rfm-y-cuartiles.pkl

-rw-r--r-- 1 dsc dsc 1M jul 15 14:22 ../Data/02.-Creacion-de-df_rfm-y-cuartiles.pkl
